# Import libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt

import os
import shutil

import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick
import numpy as np
from matplotlib import rc
from matplotlib.ticker import (
    FormatStrFormatter,
    MaxNLocator,
    MultipleLocator,
    NullFormatter,
    ScalarFormatter,
    StrMethodFormatter,
)
from scipy import interpolate, stats
from scipy.stats import norm

In [2]:
import os 
from pathlib import Path 
from glob import glob 

import matplotlib 
from matplotlib import pyplot as plt 
import seaborn as sns 

import numpy as np
import scipy 
from scipy import stats 
import pandas as pd
import itertools

# Styles

In [3]:
plt.rcParams["font.family"] = "times" # "helvetica"
plt.rcParams['axes.autolimit_mode'] = 'round_numbers'


In [4]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=0)


In [5]:
molecules = ("ethanolAA",)
engines = ("lammps-VU", "mcccs",)


# Colors
symbols = {}
symbols["cassandra"] = "o" 
symbols[ "mcccs"] = "^"
symbols["gomc"] = "s"
symbols["gromacs"] = "x" 
symbols[ "hoomd"] = "v" 
symbols[ "lammps-VU"] = "D"
#symbols[ "LAMMPS-UD"] = ">"

colors = {}
colors["cassandra"] = "#009392" #(23/256, 109/256, 156/256)
colors["mcccs"] = "#39B1B5" #(194/256, 135/256, 32/256)
colors["gomc"] = "#9CCB86" #(21/256, 138/256, 106/256)
colors["gromacs"] = "#E9E29C" #(186/256, 97/256, 26/256)
colors["hoomd"] = "#EEB479" #(193/256, 130/256, 181/256)
colors["lammps-VU"] = "#E88471" #(188/256, 146/256, 110/256)
colors["fix"] = "#FF7F00"
colors["flex"] = "#007FFF"
# colors_dict = {all_engine_molecule: all_engine_colors, 
#                pentane_fixed: pentane_fixed_colors,
#                pentane_flexible: pentane_flexible_colors}

pretty_names = {
    "mcccsflex": "MCCCS-MN-flex", 
    "mcccsfix": "MCCCS-MN-fix", 
    "lammps-VUflex": "LAMMPS-flex",
    "lammps-VUfix": "LAMMPS-fixOH",
}

fill_styles = {
    "mcccs": "none", 
    "lammps-VU": "full",
}
figsize = (9, 8)

# Load nHbonds data

In [6]:
mcccs_hbonds = list() 
for path in glob("../src/engines/mcccs/hbond_analysis_data/*"):
    mcccs_hbonds.append(Path(path))
    
lammps_hbonds = list()
for path in glob("../src/engines/lammps-VU/hbond_analysis_data/*"):
    lammps_hbonds.append(Path(path))
    
hbonds = dict((molecule, dict()) for molecule in molecules)
for lmp, mcccs in zip(sorted(mcccs_hbonds), sorted(lammps_hbonds)):
    lmp_meta = lmp.name.split("_")
    mcccs_meta= mcccs.name.split("_") 
    
    assert lmp_meta == mcccs_meta
    molecule = lmp_meta[0]
    bcond = "fix" if "NPT-fixOH" in lmp_meta[1] else "flex"
    sp = f"{lmp_meta[2]}" #f"{lmp_meta[2]}, {mcccs_meta[3]}"
        
    if sp != "400.0K":
        with open(f"{lmp.absolute()}/nhbond.txt") as f:
            lmp_hbond = np.loadtxt(f, unpack=True)    

        with open(f"{mcccs.absolute()}/nhbond.txt") as f:
            mcccs_hbond = np.loadtxt(f, unpack=True)    

        if sp not in hbonds[molecule]:
            hbonds[molecule][sp] = dict()

        hbonds[molecule][sp][bcond] ={"lammps-VU": lmp_hbond,
                                        "mcccs": mcccs_hbond}



# Combined n_hbonds, OH bond length, RDF/CDF

In [62]:
# fig, axs = plt.subplots(3, 1, sharex=False, sharey=False, figsize=(9, 15))
# ax1, ax2, ax3a = axs

fig = plt.figure(figsize=(15, 15))

gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3a = fig.add_subplot(gs[1, :])

# for ax in [ax1, ax2, ax3a]:
#     # ax.minorticks_on()
#     ax.tick_params(axis='x', which='minor', bottom=False)
#     ax.yaxis.set_major_locator(MultipleLocator(10))
#     ax.yaxis.set_minor_locator(MultipleLocator(5))
    
"""nHB Plots on ax1"""
molecule = "ethanolAA"

statepoints = sorted(set(hbonds[molecule]))
xticks = list()
sps_positions = list()
for n_sp, sp in enumerate(statepoints):
    sp_position = list()
    for engine in engines:
        for i, bcond in enumerate(["fix", "flex"]):
            try:
                ax1.errorbar(
                    1*n_sp + 0.1*i, 
                    hbonds[molecule][sp][bcond][engine][0],
                    marker=symbols[engine], 
                    yerr=hbonds[molecule][sp][bcond][engine][1], 
                    color=colors[bcond], 
                    fillstyle=fill_styles[engine],
                    ls='',
                    label=engine+bcond,)
            except:
                print(i, bcond)
            sp_position.append(1*n_sp + 0.1*i,) 
        sps_positions.append(np.mean(sp_position))
        raw_temp = sp.split('.')
        xticks.append(f'{raw_temp[0]}.0 K')


# ax1.set_xlabel("State point")
ax1.set_xticks([pos for pos in sps_positions]) 
ax1.set_xticklabels([sp for sp in xticks])
ax1.set_xlim(sps_positions[0]-0.5, sps_positions[-1]+0.5)


ax1.set_ylabel("$n\mathrm{_{HB}}$")
ax1.set_xlabel("Temperature [K]")

# wrap_labels(ax1, 10)


# get handles
handles, labels = ax1.get_legend_handles_labels()

#Sorting handles and labels: 
labels_handles = dict()
for handle, label in zip(handles, labels):
    labels_handles[label] = handle

sorted_handles = [labels_handles[engine+bcond] 
                  for engine, bcond in itertools.product(engines, ["flex", "fix"])]
sorted_labels = [pretty_names[engine+bcond] 
                 for engine, bcond in itertools.product(engines, ["flex", "fix"])]

ax1.legend(sorted_handles, 
           sorted_labels,
           facecolor='white', 
           loc ='upper right',
           ncol=2,
           prop={'size': 16},
          )
ax1.set_ylim(370, 470)


"""Plot hBond length"""

alpha = 0.2

mcccs_fix = "../src/engines/mcccs/bl_analysis_data/mcccs_ethanolAA_NPT-fixOH_280.0K_101.325kPa_cutoff_hard_lrc_energy_pressure/O_H_hist.txt"
mcccs_flex = "../src/engines/mcccs/bl_analysis_data/mcccs_ethanolAA_NPT-flexOH_280.0K_101.325kPa_cutoff_hard_lrc_energy_pressure/O_H_hist.txt"
lammps_fix = "../src/engines/mcccs/bl_analysis_data/lammps-VU_ethanolAA_NPT-fixOH_280.0K_101.325kPa_cutoff_hard_lrc_energy_pressure/O_H_hist.txt"
lammps_flex = "../src/engines/mcccs/bl_analysis_data/lammps-VU_ethanolAA_NPT-flexOH_280.0K_101.325kPa_cutoff_hard_lrc_energy_pressure/O_H_hist.txt"

mcccs_fix_data = np.genfromtxt(mcccs_fix)
mcccs_flex_data = np.genfromtxt(mcccs_flex)
lammps_fix_data = np.genfromtxt(lammps_fix)
lammps_flex_data = np.genfromtxt(lammps_flex)

ax2.plot(mcccs_flex_data[:, 0], mcccs_flex_data[:, 1], label="MCCCS-MN-flex")
ax2.plot(lammps_flex_data[:, 0], lammps_flex_data[:, 1], label="LAMMPS")
ax2.plot(lammps_fix_data[:, 0], lammps_fix_data[:, 1], label="LAMMPS-fixOH")
ax2.set_ylim([0, 20])
ax2.set_xlabel("O-H Bond length" + r" [$\mathrm{\AA}$]")

ax2.set_ylabel("Probability density")
ax2.fill_between(mcccs_flex_data[:, 0], mcccs_flex_data[:, 1], alpha=alpha)
ax2.fill_between(lammps_flex_data[:, 0], lammps_flex_data[:, 1], alpha=alpha)
ax2.fill_between(lammps_fix_data[:, 0], lammps_fix_data[:, 1], alpha=alpha)
ax2.axvline(
    0.945,
    ls="--",
    color="r",
    label="FF bond length",
)


ax2.legend(
    frameon=True,
    loc="upper right",
    ncol=1,
    # fontsize=legendfs,
    labelspacing=0.05,
)

# axes limits
ax2.set_xlim([0.85, 1.1])
ax2.set_ylim([-0.00001, 20])
ax2.yaxis.set_major_locator(plt.MaxNLocator(4))
    
    
"""RDF and CDF"""     
"""Read run files from all the independent seeds and get the final density and RDFs."""

file_flex = {}
file_rig = {}

temp = 280
file_flex[temp] = (
    f"../src/engines/mcccs/rdf_analysis_data/ethanolAA_NPT-flexOH_{temp:.1f}K_101.325kPa_cutoff_hard_lrc_energy_pressure/avg_rdf.txt"
    )
file_rig[temp] = (
    f"../src/engines/mcccs/rdf_analysis_data/ethanolAA_NPT-fixOH_{temp:.1f}K_101.325kPa_cutoff_hard_lrc_energy_pressure/avg_rdf.txt"
    )

rdf_flex = {}
rdf_rig = {}

rdf_flex[temp] = np.genfromtxt(file_flex[temp])
rdf_rig[temp] = np.genfromtxt(file_rig[temp])


ax3a.plot(
    rdf_flex[temp][:, 0], rdf_flex[temp][:, 1], label="MCCCS-MN-flex"
)
ax3a.plot(
    rdf_rig[temp][:, 0], rdf_rig[temp][:, 1], label="MCCCS-MN-fix"
)

#ax3a.grid(alpha=0.25)
# ax3a.title.set_text(r"$T = {}$".format(temp) + " K")
ax3a.set_ylim([0, 5.5])
ax3a.set_xlim([2, 6])
ax3a.yaxis.set_major_locator(plt.MaxNLocator(5))
ax3a.xaxis.set_major_locator(plt.MaxNLocator(4))

ax3a.legend(frameon=True, loc='center right')
ax3a.set_ylabel(r"$g(r)$")
ax3a.set_ylabel(r"$g(r)$")
ax3a.set_ylabel(r"$g(r)$")
ax3a.set_xlabel(r"$r$" + r" [$\mathrm{\AA}$]")


file_flex = {}
file_rig = {}

temps = [280, 300, 320]
temp = 280
file_flex[temp] = (
    f"../src/engines/mcccs/rdf_analysis_data/ethanolAA_NPT-flexOH_{temp:.1f}K_101.325kPa_cutoff_hard_lrc_energy_pressure/avg_cdf.txt"
    )

file_rig[temp] = (
     f"../src/engines/mcccs/rdf_analysis_data/ethanolAA_NPT-fixOH_{temp:.1f}K_101.325kPa_cutoff_hard_lrc_energy_pressure/avg_cdf.txt"
    )


cdf_flex = {}
cdf_rig = {}

cdf_flex[temp] = np.genfromtxt(file_flex[temp])
cdf_rig[temp] = np.genfromtxt(file_rig[temp])


ax3b = ax3a.twinx()

ax3b.plot(
    rdf_flex[temp][:, 0], cdf_flex[temp][:, 1], '--', label="MCCCS-MN-fix"
)
ax3b.plot(
    rdf_rig[temp][:, 0],
    cdf_rig[temp][:, 1],
    "--",
    label="MCCCS-MN-flex",
)

f = interpolate.interp1d(rdf_flex[temp][:, 0], cdf_flex[temp][:, 1])
g = interpolate.interp1d(rdf_rig[temp][:, 0], cdf_rig[temp][:, 1])

diff = f(3.25) - g(3.25)

# ax3b.grid(alpha=0.25)
# ax3b.title.set_text(r"$T = {}$".format(temp) + " K")
# ax3b.set_ylim([0, 2.4])
# ax3b.set_xlim([2, 4])

ax3b.set_ylim([0, 3])
ax3b.set_xlim([2.4, 4])

ax3b.yaxis.set_major_locator(plt.MaxNLocator(5))
ax3b.xaxis.set_major_locator(plt.MaxNLocator(5))
ax3b.text(
    0.8,
    0.3,
    # r"$\Delta n_{r=3.25};\mathrm{\r{A}}}$" + f"{diff:0.2f}",
    r"$\Delta n_{r=3.25\;\mathrm{\AA}}=$" + "{:.2f}".format(diff),
    
    horizontalalignment="center",
    verticalalignment="center",
    transform=ax3b.transAxes,
    fontsize=25
)


# ax3b.legend(frameon=True)
ax3b.set_ylabel(r"$n(r)$")
ax3b.set_xlabel(r"$r$" + r" [$\mathrm{\AA}$]")

ax1.yaxis.set_major_locator(MultipleLocator(20))
ax1.yaxis.set_minor_locator(MultipleLocator(10))

ax2.xaxis.set_major_locator(MultipleLocator(0.1))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.yaxis.set_major_locator(MultipleLocator(5))
ax2.yaxis.set_minor_locator(MultipleLocator(2.5))

ax3a.yaxis.set_major_locator(MultipleLocator(1))
ax3a.yaxis.set_minor_locator(MultipleLocator(0.5))

ax3b.xaxis.set_major_locator(MultipleLocator(0.4))
ax3b.xaxis.set_minor_locator(MultipleLocator(0.2))
ax3b.yaxis.set_major_locator(MultipleLocator(1))
ax3b.yaxis.set_minor_locator(MultipleLocator(0.5))

for ax in [ax1, ax2, ax3a, ax3b]:
    ax.tick_params(which='both', width=1)
    ax.tick_params(which="major", length=8, axis="y", bottom=False)
    ax.tick_params(which="minor", length=4, axis="y", bottom=False)
    
    
ax1.tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    ) # labels along the bottom edge are off
ax1.tick_params(
    axis='x',          # changes apply to the x-axis
    which='major',      # both major and minor ticks are affected
    length=8
    ) # labels along the bottom edge are off
    
for ax in [ax2, ax3a, ax3b]:
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='minor',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        length=4,
        ) # labels along the bottom edge are off
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        length=8
        ) # labels along the bottom edge are off
# for ax, label in zip([ax1, ax2, ax3a], ['(a)', '(b)', '(c)']):
#     ax.text(0.02,
#             0.925,
#             label,
#             transform=ax.transAxes, 
#             size=32,
#             weight='bold'
#            )
plt.tight_layout()
# plt.grid(alpha = 0., axis='x')
plt.savefig('figures/combined_ethanol.pdf', dpi=500, bbox_inches='tight')